In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from scipy import stats
import statsmodels.api as sm

In [95]:
df = pd.read_csv('small50000AllDataWithIdentities.csv')
df = df.loc[:, ["comment_text", "split", "toxicity", "male", "female", "LGBTQ", "christian", "muslim", "other_religions", "black", "white"]]
df = df[df['split'] == 'test']

predictions = pd.read_csv('civilcomments_split-test_seed-1_epoch-best_pred.csv', header=None)
predictions = pd.DataFrame({'predictions': np.array(predictions.values.tolist()).flatten()}, index = df.index)

df = pd.concat([predictions, df], axis=1)

df['male'] = df['male'].apply(lambda x: np.round(x>=0.5))
df['female'] = df['female'].apply(lambda x: np.round(x>=0.5))
df['LGBTQ'] = df['LGBTQ'].apply(lambda x: np.round(x>=0.5))
df['christian'] = df['christian'].apply(lambda x: np.round(x>=0.5))
df['muslim'] = df['muslim'].apply(lambda x: np.round(x>=0.5))
df['other_religions'] = df['other_religions'].apply(lambda x: np.round(x>=0.5))
df['black'] = df['black'].apply(lambda x: np.round(x>=0.5))
df['white'] = df['white'].apply(lambda x: np.round(x>=0.5))

In [26]:
print("\n")
print('Toxic samples test data: ', sum(df['toxicity']))
print('None-toxic samples test data: ', len(df['toxicity'])-sum(df['toxicity']))

print("\n")

for col in df.columns[3:]:
    print(col + ": " + str(np.sum(df[col])))

#df[(df['male'] == 0) & (df['female'] == 0) & (df['LGBTQ'] == 0) & (df['christian'] == 0) & (df['muslim'] == 0) & (df['other_religions'] == 0) & (df['black'] == 0) & (df['white'] == 0)]



Toxic samples test data:  1510.0
None-toxic samples test data:  1510.0


male: 340.0
female: 402.0
LGBTQ: 169.0
christian: 237.0
muslim: 227.0
other_religions: 102.0
black: 180.0
white: 288.0


In [111]:
def CheckAccuracy(predictions, labels):
        acc = 0.0
        for i in range(len(predictions)):
            if (predictions[i] == labels[i]):
                acc += 1
        return acc/len(predictions)

def F1AndAcc(df):
    f1_scores = []
    accuracies = []
    demographics = []
    
    for col in df.columns[4:]:
        tempdf = df[(df[col] == 1)]        
        
        labels = np.array(tempdf['toxicity'].values.tolist())
        predictions = np.array(tempdf['predictions'].values.tolist())
        
        f1_scores.append(f1_score(labels, predictions, zero_division=1))
        accuracies.append(CheckAccuracy(labels, predictions))   
    return np.array(f1_scores), np.array(accuracies)


def pRule(df):
    pRules = []
    
    for col in df.columns[3:]:
        
        tempdfz1 = df[(df[col] == 1)]      
        tempdfz0 = df[(df[col] == 0)]
        
        labelsz1 = np.array(tempdfz1['toxicity'].values.tolist())
        predictionsz1 = np.array(tempdfz1['predictions'].values.tolist())
        
        labelsz0 = np.array(tempdfz0['toxicity'].values.tolist())
        predictionsz0 = np.array(tempdfz0['predictions'].values.tolist())
        
        with np.errstate(divide='ignore'):
            
            z1Ut1 = np.sum(predictionsz1)/len(df)
            pz1 = len(predictionsz1)/len(df)
            
            z0Ut1 = np.sum(predictionsz0)/len(df)
            pz0 = len(predictionsz0)/len(df)
            
            pscore0 = (z1Ut1/pz1) / (z0Ut1/pz0)
            pscore1 = (z0Ut1/pz0) / (z1Ut1/pz1)
        
        if np.isnan(pscore0) or np.isnan(pscore1):
            finalpscore = 0
        else:
            finalpscore = min(pscore0, pscore1)
        
        pRules.append(finalpscore)
    return pRules


def pRuleOwn(df):
    pRules = []
    
    for col in df.columns[4:]:
        tempdfz1 = df[(df[col] == 1)]      
        tempdfz0 = df[(df[col] == 0)]
        
        labelsz1 = np.array(tempdfz1['toxicity'].values.tolist())
        predictionsz1 = np.array(tempdfz1['predictions'].values.tolist())
        
        labelsz0 = np.array(tempdfz0['toxicity'].values.tolist())
        predictionsz0 = np.array(tempdfz0['predictions'].values.tolist())
        
        with np.errstate(divide='ignore'):
            pscore0 = (np.sum(predictionsz1)/np.sum(labelsz1))/(np.sum(predictionsz0)/np.sum(labelsz0))
            pscore1 = (np.sum(predictionsz0)/np.sum(labelsz0))/(np.sum(predictionsz1)/np.sum(labelsz1))
        
        if np.isnan(pscore0) or np.isnan(pscore1):
            finalpscore = 0
        else:
            finalpscore = min(pscore0, pscore1)
        
        pRules.append(finalpscore)
    return pRules

def MinMaxFairness(scores):
    return np.max(scores)-np.min(scores)

def VarianceFairness(scores):
    return np.var(scores)